In [1]:
# from os import system
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
# def clear():
#     system('cls')

# Read in the original data

In [3]:
counter = 0
with open('data/ratings.csv', 'r') as f:
    df = []
    for line in f:
        counter += 1
        if counter != 1:
            items = line.split(',')
            if float(items[2]) >= 3.5: #! Filtering here on ratings below 3.5
                df.append(items[:-1])
            if (counter % 2000000) == 0:
                clear()
                print(f"evaluated {counter/1000000}M lines")

kept = len(df)

clear()
print(f"evaluated {counter} lines and kept {kept} rating events")

evaluated 2.0M lines
evaluated 4.0M lines
evaluated 6.0M lines
evaluated 8.0M lines
evaluated 10.0M lines
evaluated 12.0M lines
evaluated 14.0M lines
evaluated 16.0M lines
evaluated 18.0M lines
evaluated 20.0M lines
evaluated 22.0M lines
evaluated 24.0M lines
evaluated 25000096 lines and kept 15630129 rating events


In [4]:
df = pd.DataFrame(df, columns=["User_id", "Item_id", "Rating"])
df = df.apply(pd.to_numeric)

# Filter Data

In [5]:
# Remove all movies occuring in less than 1000 times in the dataframe
item_count = df.groupby("Item_id").size()
item_filter = item_count[item_count >= 1000].index
df = df[df["Item_id"].isin(item_filter)]

In [6]:
# Remove all users who rated less than 5 movies
user_count = df.groupby("User_id").size()
user_filter = user_count[user_count >= 5].index
df = df[df["User_id"].isin(user_filter)]
kept = len(df)
print(f"After the filters we only have {kept} Ratings")

After the filters we only have 13337921 Ratings


# Create Pivot Table

In [7]:
# Save the index so that I know the order and ID of what is fed into the NN
item_index = df.groupby("Item_id").size().index

In [8]:
df = df.pivot(index="User_id", columns="Item_id", values="Rating")
df = df.fillna(0)
df.head(5).T

User_id,1,2,3,4,5
Item_id,,,,,
1,0.0,3.5,4.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
188301,0.0,0.0,0.0,0.0,0.0
189333,0.0,0.0,0.0,4.0,0.0
192803,0.0,0.0,0.0,0.0,0.0


In [9]:
df.shape

(161288, 2540)

# Train Test Split and Save

## Dictionaries

In [10]:
id_index = dict()
index_id = dict()
counter = 0
for item_id in item_index:
    id_index.update({item_id: counter})
    index_id.update({counter: item_id})
    counter += 1

In [11]:
f = open("data/id_index.txt","w")
f.write(str(id_index))
f.close()

f = open("data/index_id.txt","w")
f.write(str(index_id))
f.close()

## ndarrays

In [12]:
# find the 90% mark in the data
tts = round(df.shape[0]*.9)
print(tts)

145159


In [13]:
train = df.sample(frac=1, random_state=73)[:tts]
test = df.sample(frac=1, random_state=73)[tts:]

In [14]:
train = train.to_numpy()
test = test.to_numpy()

In [15]:
print(train.shape)
print(train)

(145159, 2540)
[[3.5 3.5 0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [4.  0.  0.  ... 0.  0.  0. ]]


In [16]:
print(test.shape)
print(test)

(16129, 2540)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 4. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
np.save("data/train.npy", train)
np.save("data/test.npy", test)